In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
# 3000条训练数据,F1_Score:0.235,Threshold:0.5,(句向量由question中各个词的词向量取平均得到)
# 300000条训练数据,F1_Score:0.492,Threshold:0.5,(句向量由question中各个词的词向量取平均得到)
# 400000条训练数据,F1_Score:0.490,Threshold:0.5,(句向量由question中各个词的词向量取平均得到)
# 300000条训练数据,F1_Score:0.326,Threshold:0.5,(句向量由question中各个词的词向量乘以其对应的TFIDF值，再取平均得到)
# 400000条训练数据,F1_Score:0.373,Threshold:0.5,(句向量由question中各个词的词向量乘以其对应的TFIDF值，再取平均得到)
# 此版本用600000条训练数据,句向量由question中各个词的词向量取平均得到,threshold:0.5

# 之前的版本中都未将单词统一为小写，我们基于以下事实：
# 在保留单词原有形式时：
# The number of all words of train.csv and test.csv is : 172995 , The number of Unknown words is : 62346 ,The number of words with Google-vector is : 110649 
# 可以看到未在Google_vector找到词向量的单词 占 所有单词的比例高达 1/3 ，当然这是基于600000条数据做的判断；

# 在将单词统一为小写时：
# The number of all words of train.csv and test.csv is : 144678 , The number of Unknown words is : 80684 ,The number of words with Google-vector is : 63994
# 可以看到总单词量下降了3万左右，未知单词的数量增加了2万左右 ；

# 只将question_text首单词变为小写时：
# The number of all words of train.csv and test.csv is : 172917 , The number of Unknown words is : 62546 ,The number of words with Google-vector is : 110371 
# 可以看到总单词量相比保留原有单词形式时，没有太大变化；

# 此版本做如下改进：
# 1. 将question首单词变为小写； 
# 2. 句向量的构成：用各词向量各维度累加之和来表示

In [ ]:
# Data_stage_1.py
# -*- coding: utf-8 -*-
# @Time    : 2019/1/7 14:56
# @Author  : Weiyang

'''
目标：读取train.csv和test.csv数据，去除标点符号等特殊符号，将question变为word列表，单词转为小写、
输入：train.csv
输出1：questions = [[word1,word2,word3,..],...]
输出2：labels = [1,0,1,...]
输出3：all_words = [word1,word2,...] train.csv 和 test.csv 的词表,用以查找词向量以及随机生成词向量
输出4：test_questions = [[word1,word2,word3,..],...]
输出5：test_qids = [001256,05891,...]
注意:若去除特殊符号后,question为空,则将question直接删除,其不参与训练；而如果test.csv中出现此情况,则question类别确定为1,也无需参与预测;
'''

import pandas as pd
import re
import time
import numpy as np

start = time.time()
# read train.csv
train_data = pd.read_csv('../input/train.csv',encoding='utf-8-sig')

# 去除标点符号等特殊符号，将question变为词列表,将单词转为小写
questions = [] # [[word1,word2,..],..] question文本且分成一个一个词
labels = [] # labels
all_words = [] # [word1,word2,...]
symbols = ['\''] # don't, I'm
randint = np.random.randint(0,len(train_data['question_text']),100) # 产生100个随机整数
count = 0
for question,label in zip(train_data['question_text'],train_data['target']):
    if count > 600000:
        break
    if count in randint:
        print('train.csv: ',question)
    question = re.sub("[\s+\.\!\/_,\\:;><{}\-$%^*()+\"\[\]]+|[+——！，。：；》《？?、~@#￥%……&*（）]+",' ',question)
    question = [str(word) for word in question.split() if len(word.strip()) !=0 and word not in symbols]
    question = [word.rstrip('\'').lstrip('\'') for word in question] # 去除左右两侧的单引号
    question = [word.rstrip('\\').lstrip('\\') for word in question]  # 去除左右两侧的\\
    question = [word.rstrip('’').lstrip('’') for word in question]  # 去除左右两侧的’
    question = [word.rstrip('‘').lstrip('‘') for word in question]  # 去除左右两侧的‘
    question = [word.rstrip('”').lstrip('”') for word in question]  # 去除左右两侧的”
    question = [word.rstrip('“').lstrip('“') for word in question]  # 去除左右两侧的“
    question = [word.rstrip('`').lstrip('`') for word in question]  # 去除左右两侧的` 保留单词原有的形式
    #question = [word.lower() for word in question]  # 将单词统一为小写
    
    # 将question首个单词变为小写，其余单词形式不变
    temp = []
    for i in range(len(question)):
        if i == 0:
            temp.append(question[i].lower())
        else:
            temp.append(question[i])
    question = temp[:]
    
    # 判断question是否为空
    if len(question) == 0:
        continue
    all_words.extend(question)
    questions.append(question)
    labels.append(label)
    if count in randint:
        print('train.csv: ',question)
    count += 1

del train_data

# read test.csv
test_data = pd.read_csv('../input/test.csv',encoding='utf-8-sig')

# 去除标点符号等特殊符号，将question变为词列表,将单词转为小写
test_questions = [] # [[word1,word2,..],..] question文本且分成一个一个词
test_qids = [] #qids
empty_question_qids = [] # 如果去除特殊符号后,question为空,则question类别为1,即是insincere question;其不再参与预测
symbols = ['\''] # don't,I'm,...
randint = np.random.randint(0,len(test_data['question_text']),100) # 产生100个随机整数
count = 0
for question,qid in zip(test_data['question_text'],test_data['qid']):
    #if count > 3000:
        #break
    if count in randint:
        print('test.csv: ',question)
    question = re.sub("[\s+\.\!\/_,\\:;><{}\-$%^*()+\"\[\]]+|[+——！，。：；》《？?、~@#￥%……&*（）]+",' ',question)
    question = [str(word) for word in question.split() if len(word.strip()) !=0 and word not in symbols]
    question = [word.rstrip('\'').lstrip('\'') for word in question] # 去除左右两侧的单引号
    question = [word.rstrip('\\').lstrip('\\') for word in question]  # 去除左右两侧的\\
    question = [word.rstrip('’').lstrip('’') for word in question]  # 去除左右两侧的’
    question = [word.rstrip('‘').lstrip('‘') for word in question]  # 去除左右两侧的‘
    question = [word.rstrip('”').lstrip('”') for word in question]  # 去除左右两侧的”
    question = [word.rstrip('“').lstrip('“') for word in question]  # 去除左右两侧的“
    question = [word.rstrip('`').lstrip('`') for word in question]  # 去除左右两侧的` 且保留单词原有的形式不变
    #question = [word.lower() for word in question]  # 将单词统一为小写
    
    # 将question首个单词变为小写，其余单词形式不变
    temp = []
    for i in range(len(question)):
        if i == 0:
            temp.append(question[i].lower())
        else:
            temp.append(question[i])
    question = temp[:]
    
    # 判断question是否为空
    if len(question) == 0:
        empty_question_qids.append(qid)
        continue
    test_questions.append(question)
    test_qids.append(qid)
    all_words.extend(question)
    if count in randint:
        print('test.csv: ',question)
    count += 1

del test_data
all_words = set(all_words)

print('The total time of Data_stage_1.py program is : %d s' %(time.time()-start))

In [ ]:
# Data_stage_2.py
# -*- coding: utf-8 -*- 
# @Time    : 2019/1/7 16:59 
# @Author  : Weiyang

'''
数据探索
目标：探索sincere question 和 insincere question 中特征词的分布
输入：questions
#输出1：different_words = [word1,word2,...]# sincere question 和 insincere question 各自独有的词
输出2：words_idf = {word1:idf,word2:idf,...} 即找出questions中每个word的idf值，此字典可用于train.csv和test.csv
注：这里利用Gensim自动去除停用词
'''
import warnings
warnings.filterwarnings(action="ignore")
from collections import Counter
from gensim import corpora ,models
import math
import matplotlib.pyplot as plt
import numpy as np
import time

start = time.time()

##################################################  探索数据 ###########################################################
#统计question长度分布
question_len = [len(question) for question in questions]
print('The max length of question is : %d'% max(question_len))
print('The min length of question is : %d'% min(question_len))
print('The average length of question is : %d'% (sum(question_len)/float(len(question_len))))
c = Counter(question_len).most_common(100)
print('The top 100 questions length with the largest number of length are : ')
print(c)

'''
# 分别以word 的 dfs 倒序输出 insincere question 和 sincere question 中 的 1/idf ，找出与两个类别最相关的词
insincere_questions = [questions[i] for i in range(len(questions)) if labels[i] == 1]
sincere_questions = [questions[i] for i in range(len(questions)) if labels[i] == 0]

insincere_dictionary = corpora.Dictionary(insincere_questions)
id2token = {} # {id:token,..} # id:word
for token,id in insincere_dictionary.token2id.items():
    id2token[id] = token
insincere_token_frequency = {} # {word:1/idf,...},value是文档频数,指该词出现在不同question的次数/总文档的数量
for id,dfs in insincere_dictionary.dfs.items():
    insincere_token_frequency[id2token[id]] = dfs/float(len(insincere_questions))

sincere_dictionary = corpora.Dictionary(sincere_questions)
id2token = {} # {id:token,..} # id:word
for token,id in sincere_dictionary.token2id.items():
    id2token[id] = token
sincere_token_frequency = {} # {word:1/idf,...},value是文档频数指该词出现在不同question的次数/总文档的数量
for id,dfs in sincere_dictionary.dfs.items():
    sincere_token_frequency[id2token[id]] = dfs/float(len(sincere_questions))

insincere_words = [] # 存储出现在insincere question中的word ,word 以 文档频率 为准做倒排序，文档频数指该词出现在不同question的次数/总文档的数量
sincere_words = [] # 存储出现在sincere question中的word ,word 以 文档频率 为准做倒排序，文档频数指该词出现在不同question的次数/总文档的数量
ins = sorted(insincere_token_frequency.items(),key=lambda x : x[1],reverse=True)
sin = sorted(sincere_token_frequency.items(),key=lambda x : x[1],reverse=True)
#print(ins)
#print(sin)
plt.figure()
insincere_X = [] # 单词本身，用作标签
insincere_Y = [] # 单词文档频率
for item in ins:
    insincere_X.append(item[0])
    insincere_Y.append(item[1])
insincere_X = np.array(insincere_X)
insincere_Y = np.array(insincere_Y)
plt.subplot(2,1,1)
plt.bar(insincere_X[:30],insincere_Y[:30])


sincere_X = [] # 单词本身，用作标签
sincere_Y = [] # 单词文档频率
for item in sin:
    sincere_X.append(item[0])
    sincere_Y.append(item[1])
sincere_X = np.array(sincere_X)
sincere_Y = np.array(sincere_Y)
plt.subplot(2,1,2)
plt.bar(sincere_X[:30],sincere_Y[:30])
print('Print the top 30 words of insincere question and sincere question with its 1/idf...')
#plt.show()

# insincere questions 和 sincere questions 都存在的单词
share_words = set(list(insincere_X)) & set(list(sincere_X))
print(share_words)
# insincere questions 和 sincere questions 各自特殊的单词
different_words = set(list(insincere_X) + list(sincere_X)) - share_words
print(different_words)

'''
################################################# 输出word 的tfidf idf #################################################
#输出每个question中每个单词的TF-IDF值
dictionary = corpora.Dictionary(questions) # 生成字典语料
id2token = {} # {id:token,..} # id:word
for token,id in dictionary.token2id.items():
    id2token[id] = token

id_dfs = {} # {id:dfs,...} dfs是 文档频数,即有多少文档包含该单词
for id,dfs in dictionary.dfs.items():
    id_dfs[id] = dfs

corpus = [dictionary.doc2bow(question) for question in questions] # 得到每个question对应的稀疏向量(BOW向量)
model = models.TfidfModel(corpus) # 对每个question中每个单词统计其TFIDF值
vectors = model[corpus] # 将question转为BOW向量，格式为[(id,tfidf),...]
print('Starting output words_idf...')
question_num = len(questions)
words_idf = {} # {word:idf,...}
count = 0
for question_vector in vectors:
    tfidfs = {}
    idfs = {}
    for id,tfidf in question_vector:
        tfidfs[id2token[id]] = tfidf
        idf = math.log(float(question_num)/(id_dfs[id]+1),2)
        idfs[id2token[id]] = idf
        words_idf[id2token[id]] = words_idf.get(id2token[id],None)
        words_idf[id2token[id]] = idf
    # 如果question_vector没有相应word的tfidf，则其值取0(Gensim会自动去除停用词，以及删掉某些单个字符)
    # Gensim并不能对所以word都计算tfidf
    for word in questions[count]:
        if word not in tfidfs and word not in idfs:
            words_idf[word] = 10000
    count += 1
# 遍历all_words，如果该词在words_idf中没有idf值，则赋值为10000,若为0,则word代表着很强的特征意义
for word in all_words:
    if word not in words_idf:
        words_idf[word] = 10000
del dictionary,id2token,corpus,model,vectors,question_num,count,c,question_len

print('Finishing output words_idf...')
print('The total time of Data_stage_2.py program is : %d s' %(time.time()-start))

In [ ]:
# Data_stage_3.py
# -*- coding: utf-8 -*- 
# @Time    : 2019/1/8 10:03 
# @Author  : Weiyang

'''
read GoogleNews-vectors-negative300
目标：加载GoogleNews-vectors-negative300.bin词向量，如果对应词不存在词向量，则利用均匀分布随机生成一个300维的向量作词向量
输入：GoogleNews-vectors-negative300.bin(Google预训练的词向量),all_words:train.csv 和 test.csv中出现的词
输出：words_vector,通过word_vector_model[word]可以得到每个词对应的词向量,如果单词没有对应的词向量,则随机赋予一个300维的词向量
'''

import warnings
warnings.filterwarnings(action="ignore")
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
import time
import sys

# loads 300x1 word vectors from file.
def load_bin_vec(fname, vocab):
    words_vector = KeyedVectors.load_word2vec_format(fname, binary=True) # 通过model来取对应词的词向量
    return words_vector

# add random vectors of unknown words which are not in pre-trained vector file.
# if pre-trained vectors are not used, then initialize all words in vocab with random value.
def add_unknown_words(words_vector, vocab, k=300):
    unknown_words_vector = {}
    for word in vocab:
        if word not in words_vector:
            unknown_words_vector[word] = np.random.uniform(-0.25, 0.25, k)
    return unknown_words_vector

start = time.time()
# 预训练的词向量的路径
vectors_file =  '../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
print('Loading all_words ....')
print('Loading googleNew vector....')
words_vector = load_bin_vec(vectors_file, all_words)  # pre-trained vectors
unknown_words_vector = add_unknown_words(words_vector, all_words)
print('The number of all words of train.csv and test.csv is : %d , The number of Unknown words is : %d ,The number of words with Google-vector is : %d '%(len(all_words),len(unknown_words_vector),(len(all_words)-len(unknown_words_vector))))
del all_words,vectors_file,unknown_words_vector
print('The size of words_vector is : %d MB'%((sys.getsizeof(words_vector)/1024)))
#print('The size of unknown_words_vector is : %d MB'%((sys.getsizeof(unknown_words_vector)/1024)))
print('The total time of Data_stage_3.py program is : %d s' %(time.time()-start))

In [ ]:
# Gen_feature.py
# -*- coding: utf-8 -*- 
# @Time    : 2019/1/8 9:18 
# @Author  : Weiyang

'''
目标：生成句向量
输入：words_vector,questions,test_questions,questions_tfidf,words_idf
输出1：questions_vector = [[value1,value2,..],..]  question sentence vector # 将各个单词的向量累加取平均作为当前question的句向量
#输出2：questions_vector_tfidf = [[value1*tfidf,value2*tfidf,..],..] question sentence vector * tfidf  # 将各个单词的向量乘以其对应的tfidf值，再取平均作为当前question的句向量
#输出3：questions_vector_idf = [[value1*idf,value2*idf,...],..] question sentence vector * idf  # 将各个单词的向量乘以其对应的idf值，再取平均作为当前question的句向量
输出4：test_questions_vector = [[value1,value2,..],..]  question sentence vector # 将各个单词的向量累加取平均作为当前question的句向量
'''
import warnings
warnings.filterwarnings(action="ignore")
import numpy as np
from collections import Counter
import sys
import time

start = time.time()


# output question sentence vectors
print('Outputing question sentence vectors...')
questions_vector = [] 
test_questions_vector = []
for i in range(len(questions)):
    words = [word for word in questions[i]]
    vector = []
    for word in questions[i]:
        if word in words_vector:
            vector.append(words_vector[word]) # [[value1,value2,..],...]
        else:
            #vector.append(unknown_words_vector[word]) # [[value1,value2,..],...]
            vector.append(np.random.uniform(-0.25, 0.25, 300)) # 如果词向量不存在则随机赋予一个300维的词向量
    vector_lst = []
    for j in range(len(vector)):
        if words_idf[words[j]] != 10000: # 如果word的idf值存在，则word不用去除
            vector_lst.append((np.array(vector[j]) * 1).tolist())
        else:# 否则word的idf值不存在，则word是停用词，应去除
            vector_lst.append((np.array(vector[j]) * 0).tolist())
    vector = np.mat(vector_lst)
    #vector = np.mean(vector,axis=0) # 将词向量相加取平均
    vector = np.sum(vector,axis=0) # 将词向量累加
    questions_vector.append([float(value) for value in vector.tolist()[0]])

del questions
print('Del questions...')
    
for i in range(len(test_questions)):
    words = [word for word in test_questions[i]]
    vector = []
    for word in test_questions[i]:
        if word in words_vector:
            vector.append(words_vector[word]) # [[value1,value2,..],...]
        else:
            #vector.append(unknown_words_vector[word]) # [[value1,value2,..],...]
            vector.append(np.random.uniform(-0.25, 0.25, 300)) # 如果词向量不存在则随机赋予一个300维的词向量
    vector_lst = []
    for j in range(len(vector)):
        if words_idf[words[j]] != 10000: # 如果word的idf值存在，则word不用去除
            vector_lst.append((np.array(vector[j]) * 1).tolist())
        else:# 否则word的idf值不存在，则word是停用词，应去除
            vector_lst.append((np.array(vector[j]) * 0).tolist())
    vector = np.mat(vector_lst)
    #vector = np.mean(vector,axis=0) # 将词向量相加取平均
    vector = np.sum(vector,axis=0) # 将词向量累加
    test_questions_vector.append([float(value) for value in vector.tolist()[0]])


''' 
# output question sentence vectors * tfidf
print('Output question sentence vector * tfidf .... ')
questions_vector_tfidf = [] 
test_questions_vector_tfidf = []
for i in range(len(questions)):
    c = Counter(questions[i])
    words = [word for word in questions[i]]
    word_frequency = [c[word]/float(len(questions[i])) for word in questions[i]] # 计算word的词频/len(test_question)
    vector = [] # [[value1,value2,..],...]
    for word in questions[i]:
        if word in words_vector:
            vector.append(words_vector[word]) # [[value1,value2,..],...]
        else:
            #vector.append(unknown_words_vector[word]) # [[value1,value2,..],...]
            vector.append(np.random.uniform(-0.25, 0.25, 300)) # 如果词向量不存在则随机赋予一个300维的词向量
    vector = np.mat([(np.array(vector[j])*words_idf[words[j]]*float(word_frequency[j])).tolist() for j in range(len(vector))]) # 对应词乘以其tfidf值
    vector = np.mean(vector,axis=0) # 将词向量相加取平均
    #vector = np.sum(vector,axis=0) # 将词向量累加
    questions_vector_tfidf.append([float(value) for value in vector.tolist()[0]])

del questions
print('Del questions...')

for i in range(len(test_questions)):
    c = Counter(test_questions[i])
    words = [word for word in test_questions[i]]
    word_frequency = [c[word]/float(len(test_questions[i])) for word in test_questions[i]] # 计算word的词频/len(test_question)
    vector = [] # [[value1,value2,..],...]
    for word in test_questions[i]:
        if word in words_vector:
            vector.append(words_vector[word]) # [[value1,value2,..],...]
        else:
            #vector.append(unknown_words_vector[word]) # [[value1,value2,..],...]
            vector.append(np.random.uniform(-0.25, 0.25, 300)) # 如果词向量不存在则随机赋予一个300维的词向量
    vector = np.mat([(np.array(vector[j])*words_idf[words[j]]*float(word_frequency[j])).tolist() for j in range(len(vector))]) # 对应词乘以其tfidf值
    vector = np.mean(vector,axis=0) # 将词向量相加取平均
    #vector = np.sum(vector,axis=0) # 将词向量累加
    test_questions_vector_tfidf.append([float(value) for value in vector.tolist()[0]])

'''
'''    
# output question sentence vectors * idf
print('Output question sentence vector * idf .... ')
questions_vector_idf = [] 
test_questions_vector_idf = []
for i in range(len(questions)):
    words = [word for word in questions[i]]
    vector = [] # [[value1,value2,..],...]
    for word in questions[i]:
        if word in words_vector:
            vector.append(words_vector[word]) # [[value1,value2,..],...]
        else:
            #vector.append(unknown_words_vector[word]) # [[value1,value2,..],...]
            vector.append(np.random.uniform(-0.25, 0.25, 300)) # 如果词向量不存在则随机赋予一个300维的词向量
    vector = np.mat([(np.array(vector[j])*words_idf[words[j]]).tolist() for j in range(len(vector))]) # 对应词乘以其idf值
    vector = np.mean(vector,axis=0) # 将词向量相加取平均
    #vector = np.sum(vector,axis=0) # 将词向量累加
    questions_vector_idf.append([float(value) for value in vector.tolist()[0]])

del questions
print('Del questions...')

for i in range(len(test_questions)):
    words = [word for word in test_questions[i]]
    vector = [] # [[value1,value2,..],...]
    for word in test_questions[i]:
        if word in words_vector:
            vector.append(words_vector[word]) # [[value1,value2,..],...]
        else:
            #vector.append(unknown_words_vector[word]) # [[value1,value2,..],...]
            vector.append(np.random.uniform(-0.25, 0.25, 300)) # 如果词向量不存在则随机赋予一个300维的词向量
    vector = np.mat([(np.array(vector[j])*words_idf[words[j]]).tolist() for j in range(len(vector))]) # 对应词乘以其idf值
    vector = np.mean(vector,axis=0) # 将词向量相加取平均
    #vector = np.sum(vector,axis=0) # 将词向量累加
    test_questions_vector_idf.append([float(value) for value in vector.tolist()[0]])
'''
del words_vector,test_questions,words_idf
print('The total time of Gen_feature.py program is : %d s' %(time.time()-start))

In [ ]:
# xgb_model.py
# -*- coding: utf-8 -*- 
# @Time    : 2019/1/8 21:52 
# @Author  : Weiyang

'''
模块目标：用xgboost实现分类
输入1：questions_vector,test_questions_vector,labels,test_qid
输入2：questions_vector_tfidf,test_questions_vector_tfidf,labels,test_qid
输入3：questions_vector_idf,test_questions_vector_idf,labels,test_qid
'''

import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import time

class xgb_model:

    def __init__(self):
        self.data=[] # 存储vector
        self.label_true=[] # 存储label
        self.rate=0.9 #训练数据和预测数据的比例
        self.model=None #保存训练好的模型

    def load_data(self,train_data,label_data):
        """读取训练数据,格式为:value1 \t value2 \t value3 \t...\t label"""
        self.data = train_data
        self.label_true = label_data

    def train(self,is_incremental=False,batch_size=300000):
        """训练模型：由于采用增量学习的方式来训练模型,is_incremental指是否需要增量学习,batch_size指每份训练集的大小"""
        split_num=int(self.rate*len(self.data))
        x_train=self.data[:split_num]
        y_train=self.label_true[:split_num]
        num_rounds=100 #训练迭代次数
        params={
            'objective':'binary:logistic',
            'max_depth':2,
            'silent':1,
            'eta':1,
            'nthread':4
        }
        if is_incremental == True:
            # 将训练集切割成 n 份,采用增量学习的方式来训练模型
            # batch_size : 每份训练集的大小
            for start in range(0,len(x_train),batch_size):
                self.model = xgb.train(params,dtrain=xgb.DMatrix(x_train[start:start+batch_size],y_train[start:start+batch_size]),xgb_model=self.model)
        else:
            dtrain=xgb.DMatrix(x_train,y_train)
            self.model=xgb.train(params,dtrain,num_rounds,xgb_model=self.model)
        #显示重要特征,值越大说明该特征越重要
        #plot_importance(self.model)
        #plt.show()
        # 输出在测试集上的结果
        x_test = self.data[split_num:]
        y_test = self.label_true[split_num:]
        dtest = xgb.DMatrix(x_test)
        label_pre = self.model.predict(dtest)
        label_pre = list(label_pre)
        #寻找最佳阈值
        thresholds_f1 = []
        thresholds_precision = []
        thresholds_recall = []
        for thresh in np.arange(0.1,0.501,0.01):
            thresh = np.round(thresh,2)
            res_f1 = f1_score(y_test,label_pre >= thresh).astype(np.float32)
            res_precision = precision_score(y_test,label_pre >= thresh).astype(np.float32)
            res_recall = recall_score(y_test,label_pre >= thresh).astype(np.float32)
            thresholds_f1.append([thresh,res_f1])
            thresholds_precision.append([thresh, res_precision])
            thresholds_recall.append([thresh, res_recall])
            print("Threshold:%.4f , Precision:%.4f , Recall:%.4f ,F1:%.4f " % (thresh,precision_score(y_test, label_pre >= thresh), recall_score(y_test, label_pre >= thresh),f1_score(y_test, label_pre >= thresh)))   
        print('*'*100)
        thresholds_f1.sort(key=lambda x : x[1],reverse=True)
        best_thresh = thresholds_f1[0][0]
        print('Best F1 Score at threshold {0} is {1}'.format(best_thresh,thresholds_f1[0][1]))

        thresholds_precision.sort(key=lambda x: x[1], reverse=True)
        best_thresh = thresholds_precision[0][0]
        print('Best Precision at threshold {0} is {1}'.format(best_thresh, thresholds_precision[0][1]))

        thresholds_recall.sort(key=lambda x: x[1], reverse=True)
        best_thresh = thresholds_recall[0][0]
        print('Best Recall at threshold {0} is {1}'.format(best_thresh, thresholds_recall[0][1]))

    def cross_validation(self):
        """交叉验证训练模型"""
        x_train = self.data
        y_train = self.label_true
        dtrain = xgb.DMatrix(x_train, y_train)
        num_rounds = 500  # 训练迭代次数
        params={
            'objective':'binary:logistic',
            'max_depth':2,
            'silent':0,
            'eta':1,
            'nthread':4
        }
        result=xgb.cv(params,dtrain,num_rounds,nfold=10)#10折交叉验证,全部数据
        for line in result:
            print(line)

    def predict(self,test_data,test_qid,threshold):
        """模型预测:test_data_path:测试数据路径,model_path:加载训练好的模型,result_path:测试结果保存路径,threshold阈值，大于等于则为label 1 小于则为 label 0"""
        #读取训练数据,格式为:qid \t value1 \t value2 \t....
        test_data = test_data # 存储question的vector
        test_qid = test_qid # 存储question的qid
        dtest=xgb.DMatrix(test_data)
        # load trained model
        label_pre=self.model.predict(dtest)
        label_pre = [int(prob >= threshold) for prob in list(label_pre)]
        # 输出预测结果
        result = pd.DataFrame(columns=['qid','prediction'])
        result['qid'] = test_qid
        result['prediction'] = label_pre
        #print(label_pre)
        return result

start = time.time()
x = xgb_model()
# 训练模型
print(len(questions_vector),len(labels))
x.load_data(questions_vector,labels)
x.train(is_incremental=True,batch_size=600000) #增量学习,每份训练集包含30万条数据
#x.cross_validation()
#del questions_vector,labels
# 模型预测
sub = x.predict(test_questions_vector,test_qids,0.26)
# 将question为空的qid的类别添加进去并且设置为1
if len(empty_question_qids) !=0:
    empty_questions = pd.DataFrame(columns=['qid','prediction'])
    empty_questions['qid'] = empty_question_qids
    empty_questions['prediction'] = [1]*len(empty_question_qids)
    sub = pd.concat([sub,empty_questions],axis=0)
sub.to_csv('submission.csv',index=False)
print(empty_question_qids)

print('The total time of xgb_model.py program is : %d min' % ((time.time() - start)/60))